Пример, который обучается генерировать тексты похожие на тексты Ницце. 

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using Theano backend.
DEBUG: nvcc STDOUT nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mod.cu
C:\Users\sanek\Anaconda3\lib\site-packages\theano-0.8.2-py3.5.egg\theano\sandbox\cuda\cnmem.cpp(668): warning C4477: "fprintf": ў бва®ЄҐ д®а¬ вЁа®ў ­Ёп "%lu" ваҐЎгҐвбп  аЈг¬Ґ­в вЁЇ  "unsigned long", ­® ў аЁ вЁў­л©  аЈг¬Ґ­в "2" Ё¬ҐҐв вЁЇ "std::size_t"
C:\Users\sanek\Anaconda3\lib\site-packages\theano-0.8.2-py3.5.egg\theano\sandbox\cuda\cnmem.cpp(668): note: аҐЄ®¬Ґ­¤гҐвбп ЁбЇ®«м§®ў вм "%zu" ў бва®ЄҐ д®а¬ вЁа®ў ­Ёп
C:\Users\sanek\Anaconda3\lib\site-packages\theano-0.8.2-py3.5.egg\theano\sandbox\cuda\cnmem.cpp(670): warning C4477: "fprintf": ў бва®ЄҐ д®а¬ вЁа®ў ­Ёп "%016lx" ваҐЎгҐвбп  аЈг¬Ґ­в вЁЇ  "unsigned long", ­® ў аЁ вЁў­л©  аЈг¬Ґ­в "1" Ё¬ҐҐв вЁЇ "std::size_t"
C:\Users\sanek\Anaconda3\lib\site-packages\theano-0.8.2-py3.5.egg\theano\sandbox\cuda\cnmem.cpp(670): no

Скачиваем текст 

In [2]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

598016/600901 [============================>.] - ETA: 0scorpus length: 600901


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 59
nb sequences: 200287
Vectorization...


In [4]:
X.shape

(200287, 40, 59)

Keras довольно высокоуровневый интерфейс. 
Для построения модели достаточно просто описать какие слои вы хотите и в каком порядке. 

In [5]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


Обучение работает очень долго. Имеет смысл дождаться хотя бы одной итерации и попытаться сгененировать текст. 


In [ ]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()